In [59]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
using Test
using Plots
include("quadratic_cost.jl")
include("walker.jl")
include("simple_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")

  Activating environment at `~/Desktop/16745_OCRL/Project/Optimal-Control-Project/Project.toml`
Precompiling project...
  ✓ Wayland_protocols_jll
  ✓ OpenSSL_jll
  ✓ LERC_jll
  ✓ xkbcommon_jll
  ✓ ChainRulesCore
  ✓ ArrayInterface
  ✓ Libtiff_jll
  ✓ FFMPEG_jll
  ✓ StructArrays
  ✓ Qt5Base_jll
  ✓ FFMPEG
  ✓ ChangesOfVariables
  ✓ WebIO
  ✓ GR_jll
  ✓ FiniteDiff
  ✓ LogExpFunctions
  ✓ PlotUtils
  ✓ JSExpr
  ✓ OSQP
  ✓ Interpolations
  ✓ GR
  ✓ StatsBase
  ✓ Blink
  ✓ RecipesPipeline
  ✓ SpecialFunctions
  ✓ DiffRules
  ✓ GeometryBasics
  ✓ PlotThemes
  ✓ MeshIO
  ✓ ForwardDiff
  ✓ MeshCat
  ✓ SparseDiffTools
  ✓ RobotDynamics
  ✓ RobotZoo
  ✓ TrajectoryOptimization
  ✓ TrajOptPlots
  ✓ Altro
  ✓ Plots
  38 dependencies successfully precompiled in 38 seconds (164 already precompiled)
  30 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


slerp (generic function with 1 method)

In [51]:
model = SimpleQuadruped()
n,m = state_dim(model), control_dim(model)
# mvis = initialize_visualizer(model)
# render(mvis)

(37, 12)

## Cost Fxn

Stealing from HW4, quadractic_cost.jl

$$ \begin{bmatrix}
c_\text{init} \\
c_\text{goal} \\
c_\text{dynamics} \\
c_\text{stance} \\
c_\text{length} \\
\end{bmatrix}$$
which are of length $n$, $n$, $(N-1)n$, $N$, and $2N$, respectively. The dynamics, stance, and length constraints should be ordered by time step.

In [50]:
"""
    HybridNLP{n,m,L,Q}

Represents a (N)on(L)inear (P)rogram of a trajectory optimization problem,
with a dynamics model of type `L`, a quadratic cost function, horizon `T`, 
and initial and final state `x0`, `xf`.

The kth state and control can be extracted from the concatenated state vector `Z` using
`Z[nlp.xinds[k]]`, and `Z[nlp.uinds[k]]`.

# Constructor
    HybridNLP(model, obj, tf, N, M, x0, xf, [integration])

# Basic Methods
    Base.size(nlp)    # returns (n,m,T)
    num_ineq(nlp)     # number of inequality constraints
    num_eq(nlp)       # number of equality constraints
    num_primals(nlp)  # number of primal variables
    num_duals(nlp)    # total number of dual variables
    packZ(nlp, X, U)  # Stacks state `X` and controls `U` into one vector `Z`

# Evaluating the NLP
The NLP supports the following API for evaluating various pieces of the NLP:

    eval_f(nlp, Z)         # evaluate the objective
    grad_f!(nlp, grad, Z)  # gradient of the objective
    eval_c!(nlp, c, Z)     # evaluate the constraints
    jac_c!(nlp, c, Z)      # constraint Jacobian
"""
struct HybridNLP{n,m,L,Q} <: MOI.AbstractNLPEvaluator
    model::L                                 # dynamics model
    obj::Vector{QuadraticCost{n,m,Float64}}  # objective function
    N::Int                                   # number of knot points
    M::Int                                   # number of steps in each mode
    Nmodes::Int                              # number of modes
    tf::Float64                              # total time (sec)
    x0::MVector{n,Float64}                   # initial condition
    xf::MVector{n,Float64}                   # final condition
    times::Vector{Float64}                   # vector of times
    modes::Vector{Int}                       # mode ID
    xinds::Vector{SVector{n,Int}}            # Z[xinds[k]] gives states for time step k
    uinds::Vector{SVector{m,Int}}            # Z[uinds[k]] gives controls for time step k
    cinds::Vector{UnitRange{Int}}            # indices for each of the constraints
    lb::Vector{Float64}                      # lower bounds on the constraints
    ub::Vector{Float64}                      # upper bounds on the constraints
    zL::Vector{Float64}                      # lower bounds on the primal variables
    zU::Vector{Float64}                      # upper bounds on the primal variables
    rows::Vector{Int}                        # rows for Jacobian sparsity
    cols::Vector{Int}                        # columns for Jacobian sparsity
    use_sparse_jacobian::Bool
    blocks::BlockViews
    function HybridNLP(model, obj::Vector{<:QuadraticCost{n,m}},
            tf::Real, N::Integer, M::Integer, x0::AbstractVector, xf::AbstractVector, 
            integration::Type{<:QuadratureRule}=RK4; use_sparse_jacobian::Bool=false
        ) where {n,m}
        # Create indices
        xinds = [SVector{n}((k-1)*(n+m) .+ (1:n)) for k = 1:N]
        uinds = [SVector{m}((k-1)*(n+m) .+ (n+1:n+m)) for k = 1:N-1]
        times = collect(range(0, tf, length=N))
        
        # Specify the mode sequence
        
        # NOTE: MODIFY THIS MODE SEQUENCE ####################
        modes = map(1:N) do k
            isodd((k-1) ÷ M + 1) ? 1 : 2
        end
        Nmodes = Int(ceil(N/M))
        # ####################################################
        
        # specify the constraint indices
        c_init_inds = 1:n                                                  # initial constraint
        c_term_inds = (c_init_inds[end]+1):(c_init_inds[end]+n)            # terminal constraint
        c_dyn_inds = (c_term_inds[end]+1):(c_term_inds[end]+n*(N-1))       # dynamics constraints
        c_length_inds = (c_dyn_inds[end]+1):(c_dyn_inds[end]+(4*N))        # length bounds     (4 per time step)
        c_height_inds = (c_length_inds[end]+1):(c_length_inds[end]+(4*N))  # stance foot height = ground, swing foot height > ground (4 per time step)
        
        m_nlp = c_length_inds.stop # total number of constraints
        
        
        # TODO: specify the bounds on the constraints
        #lb = fill(+Inf,m_nlp)                                                # lower bounds on the constraints
        #ub = fill(-Inf,m_nlp)                                                # upper bounds on the constraints
        
        lb = zeros(m_nlp)
        lb[c_length_inds] .= 0.5 # min length
        
        ub = zeros(m_nlp)
        ub[c_length_inds] .= 1.5 # max length
        

        
        # Other initialization
        cinds = [c_init_inds, c_term_inds, c_dyn_inds, c_length_inds, c_height_inds]
        n_nlp = n*N + (N-1)*m
        zL = fill(-Inf, n_nlp)
        zU = fill(+Inf, n_nlp)
        rows = Int[]
        cols = Int[]
        blocks = BlockViews(m_nlp, n_nlp)
        
        new{n,m,typeof(model), integration}(
            model, obj,
            N, M, Nmodes, tf, x0, xf, times, modes,
            xinds, uinds, cinds, lb, ub, zL, zU, rows, cols, use_sparse_jacobian, blocks
        )
    end
end
Base.size(nlp::HybridNLP{n,m}) where {n,m} = (n,m,nlp.N)
num_primals(nlp::HybridNLP{n,m}) where {n,m} = n*nlp.N + m*(nlp.N-1)
num_duals(nlp::HybridNLP) = nlp.cinds[end][end]

"""
    packZ(nlp, X, U)

Take a vector state vectors `X` and controls `U` and stack them into a single vector Z.
"""
function packZ(nlp, X, U)
    Z = zeros(num_primals(nlp))
    for k = 1:nlp.N-1
        Z[nlp.xinds[k]] = X[k]
        Z[nlp.uinds[k]] = U[k]
    end
    Z[nlp.xinds[end]] = X[end]
    return Z
end

"""
    unpackZ(nlp, Z)

Take a vector of all the states and controls and return a vector of state vectors `X` and
controls `U`.
"""
function unpackZ(nlp, Z)
    X = [Z[xi] for xi in nlp.xinds]
    U = [Z[ui] for ui in nlp.uinds]
    return X, U
end

function TrajOptPlots.visualize!(vis, nlp::HybridNLP, Z)
    TrajOptPlots.visualize!(vis, nlp.model, nlp.tf, unpackZ(nlp, Z)[1])
end

In [ ]:
# includes the interface to Ipopt
include("moi.jl")

In [44]:
"""
    eval_f(nlp, Z)

Evaluate the objective, returning a scalar.
"""
function eval_f(nlp::HybridNLP, Z)
    J = 0.0
    xi,ui = nlp.xinds, nlp.uinds
    for k = 1:nlp.N-1
        x,u = Z[xi[k]], Z[ui[k]]
        J += stagecost(nlp.obj[k], x, u)
    end
    J += termcost(nlp.obj[end], Z[xi[end]])
    return J
end

"""
    grad_f!(nlp, grad, Z)

Evaluate the gradient of the objective at `Z`, storing the result in `grad`.
"""
function grad_f!(nlp::HybridNLP{n,m}, grad, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    obj = nlp.obj
    for k = 1:nlp.N-1
        x,u = Z[xi[k]], Z[ui[k]]
        grad[xi[k]] = obj[k].Q*x + obj[k].q
        grad[ui[k]] = obj[k].R*u + obj[k].r
    end
    grad[xi[end]] = obj[end].Q*Z[xi[end]] + obj[end].q
    return nothing
end

grad_f!

## Reference Trajectory

In [53]:
function reference_trajectory(model::SimpleQuadruped, times;
        xinit = 0.0,
        xterm = 10.0*model,
        height = 0.8, # MODIFY LATER based on leg lengths
    )
    
    # Some useful variables
    n,m = size(model)
    tf = times[end]
    N = length(times)
    Δx = xterm - xinit
    
    mb,g = model.mb, model.g
    body_width = 2*abs(model.s1[2]) # from FL shoulder y pos wrt COM
    body_length = 2*abs(model.s1[1]) # from FL shoulder x pos wrt COM
    
    # initialization
    xref = zeros(n,N)
    uref = zeros(m,N)
    
    # linearly interpolate x-pos
    # constant y-pos
    # velocities by taking difference
    
    xs = range(xinit,xterm,length=N)
    
    dt = Δx / tf
    
    # do this smarter in the future
    # to make a less boring trajectory
    
    for k = 1:N-1
          
        xref[1,k] = xs[k] # body x
        xref[2,k] = 0 # body y
        xref[3,k] = height # body z
        
        # no body rotation in reference
        
        # foot index: (1,2,3,4) = (FL,BL,FR,BR)
        
        # foot x pos
        xref[8,k] = xs[k] + (body_length/2)
        xref[11,k] = xs[k] - (body_length/2)
        xref[14,k] = xs[k] + (body_length/2)
        xref[17,k] = xs[k] - (body_length/2)
        
        # foot y pos
        xref[9,k] = -body_width / 2
        xref[12,k] = -body_width / 2
        xref[15,k] = body_width / 2
        xref[18,k] = body_width / 2
        
        # foot z pos
        # keep at 0

        xref[20,k] = (xs[k+1] - xs[k])/dt # body x vel
        xref[26,k] = (xs[k+1] - xs[k])/dt # foot1 x vel
        xref[29,k] = (xs[k+1] - xs[k])/dt # foot2 x vel
        xref[32,k] = (xs[k+1] - xs[k])/dt # foot3 x vel
        xref[35,k] = (xs[k+1] - xs[k])/dt # foot4 x vel
    end
    
    # end state
    # set terminal positions, all velocities should 0 out
    
    # foot x pos
    xref[8,N] = xterm + (body_length/2)
    xref[11,N] = xterm - (body_length/2)
    xref[14,N] = xterm + (body_length/2)
    xref[17,N] = xterm - (body_length/2)

    # foot y pos
    xref[9,N] = -body_width / 2
    xref[12,N] = -body_width / 2
    xref[15,N] = body_width / 2
    xref[18,N] = body_width / 2
    
    
    # reference trajectory
    uref .= kron(ones(N)', [0.5*mb*g; 0.5*mb*g; 0.5*mb*g; 0.5*mb*g]) # 1/2mg for each foot
    
    # Convert to a trajectory
    Xref = [SVector{n}(x) for x in eachcol(xref)]
    Uref = [SVector{m}(u) for u in eachcol(uref)]
    return Xref, Uref
end

reference_trajectory (generic function with 2 methods)

In [54]:
# Discretization
tf = 4.4
dt = 0.1
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N);

# Reference Trajectory
Xref,Uref = reference_trajectory(model, times)

# ALL THE MODEL PARAMS ARE WRONG
# so nothing will actually work rn

LoadError: type SimpleQuadruped has no field body_width

In [58]:
body_length = 2*abs(model.s1[1])

2.0

## Contact Sequence / Gait

In [29]:
# Discretization
tf = 4.4
dt = 0.1
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N);

function contact_sequence_trot(times)
    # 1 = stance, 0 = swing
    # 4xN matrix encoding contact schedule
    
    # initialize
    contact_schedule = zeros(4,length(times))

    step_length = 5 # NOTE: check later, is this physically feasible w/ our dt

    #hardcoded a trot
    for k = 1:N
        if k % (2*step_length) < step_length
            contact_schedule[:,k] .= [1;0;0;1]
        else
            contact_schedule[:,k] .= [0;1;1;0]
        end
    end
    return contact_schedule
end

contact_sched = contact_sequence_trot(times)

4×45 Matrix{Float64}:
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  …  0.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0     0.0  1.0  1.0  1.0  1.0  1.0  0.0